In [9]:
import torch
import torchvision
import h5py
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.backends.cudnn as cudnn
import numpy as np
deep = True
if deep:
    from DEEP_ECE_CNN import CNN
else:
    from SHALLOW_ECE_CNN import CNN

In [10]:
#Load in the data
from load_data import EEGDataLoader
data = EEGDataLoader('project_datasets/')
X_train, y_train, X_test, y_test =  data.load_all_data()

(288, 22, 1000)
(288, 22, 1000)
(288, 22, 1000)
(288, 22, 1000)
(288, 22, 1000)
(288, 22, 1000)
(288, 22, 1000)
(288, 22, 1000)
(288, 22, 1000)


In [11]:
#Initialize the CNN
cuda = False
if torch.cuda.is_available():
    cuda = True
def CNN_init(lr):
    cnn = CNN()
    if torch.cuda.is_available():
        cnn.cuda()
        cnn = torch.nn.DataParallel(cnn,device_ids=range(torch.cuda.device_count()))
        torch.backends.cudnn.benchmark = True
    lr = lr
    optimizer = torch.optim.Adam(cnn.parameters(),lr=lr)
    criterion = nn.CrossEntropyLoss().cuda()
    return cnn, optimizer, criterion

In [12]:
def train(cnn, X_train, y_train, epoch, batch_size, cuda, optimizer, criterion,X_test,y_test):
    X_train_i, y_train_i = X_train[0], y_train[0]
    show_acc = True
    for epoch in range(epoch):
        print('Epoch: ',epoch)
        random = np.random.choice(X_train_i.shape[0],batch_size)#,replace=False)
        iter = 0
        for i in random:
            image, label = X_train_i[i], y_train_i[i]
            if np.any(np.isnan(image)):
                continue
            if cuda:
                image = autograd.Variable(torch.cuda.FloatTensor(image.reshape((image.shape[0],1,image.shape[1]))))
                label = autograd.Variable(torch.cuda.LongTensor([int(label %769)]))
            else:
                image = autograd.Variable(torch.FloatTensor(image.reshape((image.shape[0],1,image.shape[1]))))
                label = autograd.Variable(torch.LongTensor([int(label %769)]))
            optimizer.zero_grad()
            scores = cnn(image)
            scores = torch.mean(scores,0)
            scores = scores.view(1,4)
            loss = criterion(scores,label)
            loss.backward()
            optimizer.step()
        if show_acc:
            print('Train Accuracy: ',train_acc(cnn,X_train,y_train,cuda,optimizer))
            print('Test Accuracy: ',test(cnn,X_test,y_test,cuda,optimizer))
       # print('Loss: ',loss.data[0])

In [13]:
# Training Accuracy
def train_acc(cnn, X_train, y_train, cuda, optimizer):
    predictions = []
    labels = []
    X_train_i, y_train_i = X_train[0], y_train[0]
    for i in range(X_train_i.shape[0]):
        image, label = X_train_i[i], y_train_i[i]
        if np.any(np.isnan(image)):
            continue
        if cuda:
            image = autograd.Variable(torch.cuda.FloatTensor(image.reshape((image.shape[0],1,image.shape[1]))))
            label = autograd.Variable(torch.cuda.LongTensor([int(label %769)]))
        else:
            image = autograd.Variable(torch.FloatTensor(image.reshape((image.shape[0],1,image.shape[1]))))
            label = autograd.Variable(torch.LongTensor([int(label %769)]))
        optimizer.zero_grad()
        scores = cnn(image)
        scores = torch.mean(scores,0)
        scores = scores.view(1,4)
        prediction = np.argmax(scores.data.cpu().numpy())
        predictions.append(prediction)
        labels.append(int(label%769))
    diffs = [1 if prediction != label else 0 for prediction, label in zip(predictions, labels)]
    error = sum(diffs)/len(diffs)
    #print("accuracy: {}".format(1 - error))
    return (1 - error)

In [ ]:
# Testing Accuracy
def test(cnn, X_test, y_test, cuda, optimizer):
    X_test_i, y_test_i = X_test[0], y_test[0]
    predictions = []
    labels = []
    for i in range(X_test_i.shape[0]):
        image, label = X_test_i[i], y_test_i[i]
        if np.any(np.isnan(image)):
            continue
        if cuda:
            image = autograd.Variable(torch.cuda.FloatTensor(image.reshape((image.shape[0],1,image.shape[1]))))
            label = autograd.Variable(torch.cuda.LongTensor([int(label %769)]))
        else:
            image = autograd.Variable(torch.FloatTensor(image.reshape((image.shape[0],1,image.shape[1]))))
            label = autograd.Variable(torch.LongTensor([int(label %769)]))
        optimizer.zero_grad()
        scores = cnn(image)
        scores = torch.mean(scores,0)
        scores = scores.view(1,4)
        prediction = np.argmax(scores.data.cpu().numpy())
        predictions.append(prediction)
        labels.append(int(label%769))


    diffs = [1 if prediction != label else 0 for prediction, label in zip(predictions, labels)]
    error = sum(diffs)/len(diffs)
    #print("accuracy: {}".format(1 - error))
    return (1 - error)

In [ ]:
lr = 0.0001
ep = 40
batch = 238
cnn, opt, crit = CNN_init(lr)
train(cnn, X_train, y_train, ep, batch, cuda, opt, crit,X_test,y_test)
print('Learning Rate: ', lr, ' Batch Size: ', batch,' Epochs: ', ep)
train_a = train_acc(cnn,X_train,y_train, cuda, opt)
test_a = test(cnn,X_test,y_test, cuda, opt)
print('Train: ', train_a, ' Test: ', test_a )

In [ ]:
# test a lot of shit
learning_rates = np.array([0.1,5e-2,1e-2,5e-3,1e-3,5e-4,1e-4,5e-5,1e-5])
batch_sizes = np.array([238,200,150,100,50])
epochs = np.array([1,5,10,20,40])
data = np.zeros((learning_rates.shape[0],batch_sizes.shape[0],epochs.shape[0],2))
l = 0
for lr in learning_rates:
    b = 0
    for batch in batch_sizes:
        e = 0
        prev_train = 0
        for ep in epochs:
            if prev_train == 1:
                continue
            cnn, opt, crit = CNN_init(lr)
            train(cnn, X_train, y_train, ep, batch, cuda, opt, crit,X_test,y_test)
            print('Learning Rate: ', lr, ' Batch Size: ', batch,' Epochs: ', ep)
            train_a = train_acc(cnn,X_train,y_train, cuda, opt)
            test_a = test(cnn,X_test,y_test, cuda, opt)
            print('Train: ', train_a, ' Test: ', test_a )
            data[l,b,e,0] = train_a
            data[l,b,e,1] = test_a
            prev_train = train_a
            e += 1
        b += 1
    l += 1

In [ ]:
def train_crop(cnn, X_train, y_train, epoch, batch_size, cuda, optimizer, criterion):
    X_train_i, y_train_i = X_train[0], y_train[0]
    for epoch in range(epoch):
        print('Epoch: ',epoch)
        iter = 0
        random = np.random.choice(X_train_i.shape[0],batch_size)
        X_train_i = X_train[0][random]
        for i in range(X_train_i.shape[0]):
            image, label = X_train_i[i], y_train_i[i]
            if np.any(np.isnan(image)):
                    continue
            for j in range(10):
                lower = j*100
                upper = (j+1)*100 - 1
                image_c = image[:,lower:upper]
                if cuda:
                    image_c = autograd.Variable(torch.cuda.FloatTensor(image.reshape((image.shape[0],1,image.shape[1]))))
                    label = autograd.Variable(torch.cuda.LongTensor([int(label %769)]))
                else:
                    image_c = autograd.Variable(torch.FloatTensor(image.reshape((image.shape[0],1,image.shape[1]))))
                    label = autograd.Variable(torch.LongTensor([int(label %769)]))
                optimizer.zero_grad()
                scores = cnn(image_c)
                scores = torch.mean(scores,0)
                scores = scores.view(1,4)
                loss = criterion(scores,label)
                loss.backward()
                optimizer.step()

In [ ]:
def acc_crop(cnn, X_train, y_train, cuda, optimizer):
    X_train_i, y_train_i = X_train[0], y_train[0]
    labels = []
    predictions = []
    for i in range(X_train_i.shape[0]):
        image, label = X_train_i[i], y_train_i[i]
        if np.any(np.isnan(image)):
                continue
        scores_t = np.zeros(4)
        if cuda:
            scores_t = autograd.Variable(torch.cuda.FloatTensor(scores_t.reshape(1,4)))
        else:
            scores_t = autograd.Variable(torch.FloatTensor(scores_t.reshape(1,4)))
        for j in range(10):
            lower = j*100
            upper = (j+1)*100 - 1
            image_c = image[:,lower:upper]
            if cuda:
                image_c = autograd.Variable(torch.cuda.FloatTensor(image.reshape((image.shape[0],1,image.shape[1]))))
                label = autograd.Variable(torch.cuda.LongTensor([int(label %769)]))
            else:
                image_c = autograd.Variable(torch.FloatTensor(image.reshape((image.shape[0],1,image.shape[1]))))
                label = autograd.Variable(torch.LongTensor([int(label %769)]))
            optimizer.zero_grad()
            scores = cnn(image_c)
            scores = torch.mean(scores,0)
            scores = scores.view(1,4)
            scores_t += scores
        prediction = np.argmax(scores_t.data.cpu().numpy())
        predictions.append(prediction)
        labels.append(int(label%769))
    diffs = [1 if prediction != label else 0 for prediction, label in zip(predictions, labels)]
    error = sum(diffs)/len(diffs)
    print("accuracy: {}".format(1 - error))
    return (1 - error)

In [ ]:
# Test cropped
lr = 0.001
ep = 10
batch = 150
cnn, opt, crit = CNN_init(lr)
train_crop(cnn, X_train, y_train, ep, batch, cuda, opt, crit)
print('Learning Rate: ', lr, ' Batch Size: ', batch,' Epochs: ', ep)
train_a = acc_crop(cnn,X_train,y_train, cuda, opt)
test_a = acc_crop(cnn,X_test,y_test, cuda, opt)
print('Train: ', train_a, ' Test: ', test_a )

In [ ]:
train_a = acc_crop(cnn,X_train,y_train, cuda, opt)
test_a = acc_crop(cnn,X_test,y_test, cuda, opt)
print('Train: ', train_a, ' Test: ', test_a )

In [5]:
#Initialize the CNN
cnn = CNN()
cuda = False
if torch.cuda.is_available():
    print('Using GPU')
    cuda = True
    cnn.cuda()
    cnn = torch.nn.DataParallel(cnn,device_ids=range(torch.cuda.device_count()))
    torch.backends.cudnn.benchmark = True
lr = 0.001
epochs = 50
batch_size = 150
optimizer = torch.optim.Adam(cnn.parameters(),lr=lr)
criterion = nn.CrossEntropyLoss().cuda()

Using GPU


In [6]:
#train the data on all subjects
for epoch in range(epochs):
    print('Epoch: ',epoch)
    for k in range(X_train.shape[0]):
        X_train_i, y_train_i = X_train[k], y_train[k]
        print('Subject: ', k+1)
        for i in range(X_train_i.shape[0]):
            image, label = X_train_i[i], y_train_i[i]
            if np.any(np.isnan(image)):
                continue
            if cuda:
                image = autograd.Variable(torch.cuda.FloatTensor(image.reshape((image.shape[0],1,image.shape[1]))))
                label = autograd.Variable(torch.cuda.LongTensor([int(label %769)]))
            else:
                image = autograd.Variable(torch.FloatTensor(image.reshape((image.shape[0],1,image.shape[1]))))
                label = autograd.Variable(torch.LongTensor([int(label %769)]))
            optimizer.zero_grad()
            scores = cnn(image)
            scores = torch.mean(scores,0)
            scores = scores.view(1,4)
            loss = criterion(scores,label)
            loss.backward()
            optimizer.step()

        print('Loss: ',loss.data[0])

Epoch:  0
Subject:  1
Loss:  1.6033002138137817
Subject:  2
Loss:  1.5818686485290527
Subject:  3
Loss:  2.2883572578430176
Subject:  4
Loss:  1.5322730541229248
Subject:  5
Loss:  0.6604452729225159
Subject:  6
Loss:  1.0602604150772095
Subject:  7
Loss:  1.4509241580963135
Subject:  8
Loss:  1.8742921352386475
Subject:  9
Loss:  1.1256470680236816
Epoch:  1
Subject:  1
Loss:  1.2024261951446533
Subject:  2
Loss:  0.9311540126800537
Subject:  3
Loss:  1.5646218061447144
Subject:  4
Loss:  1.7788896560668945
Subject:  5
Loss:  1.184976577758789
Subject:  6
Loss:  1.1306202411651611
Subject:  7
Loss:  1.100861668586731
Subject:  8
Loss:  1.4377527236938477
Subject:  9
Loss:  1.3461034297943115
Epoch:  2
Subject:  1
Loss:  1.490314245223999
Subject:  2
Loss:  1.1507220268249512
Subject:  3
Loss:  1.3347386121749878
Subject:  4
Loss:  1.8304784297943115
Subject:  5
Loss:  0.9821665287017822
Subject:  6
Loss:  1.0596559047698975
Subject:  7
Loss:  1.4140074253082275
Subject:  8
Loss:  1.84

Loss:  0.31536662578582764
Epoch:  23
Subject:  1
Loss:  0.3397812843322754
Subject:  2
Loss:  7.534027099609375e-05
Subject:  3
Loss:  0.0037322044372558594
Subject:  4
Loss:  0.043067216873168945
Subject:  5
Loss:  0.0002536773681640625
Subject:  6
Loss:  0.30431485176086426
Subject:  7
Loss:  0.00023794174194335938
Subject:  8
Loss:  0.01714944839477539
Subject:  9
Loss:  0.03368997573852539
Epoch:  24
Subject:  1
Loss:  0.01490163803100586
Subject:  2
Loss:  9.5367431640625e-06
Subject:  3
Loss:  0.000152587890625
Subject:  4
Loss:  0.06446433067321777
Subject:  5
Loss:  0.003097057342529297
Subject:  6
Loss:  0.04683327674865723
Subject:  7
Loss:  0.0007491111755371094
Subject:  8
Loss:  0.00011157989501953125
Subject:  9
Loss:  0.07625389099121094
Epoch:  25
Subject:  1
Loss:  0.0030546188354492188
Subject:  2
Loss:  0.000194549560546875
Subject:  3
Loss:  2.384185791015625e-05
Subject:  4
Loss:  0.007811307907104492
Subject:  5
Loss:  0.000919342041015625
Subject:  6
Loss:  0.03

Loss:  5.7220458984375e-06
Subject:  2
Loss:  0.0
Subject:  3
Loss:  0.0029611587524414062
Subject:  4
Loss:  0.2381420135498047
Subject:  5
Loss:  0.0
Subject:  6
Loss:  0.0075836181640625
Subject:  7
Loss:  0.0008778572082519531
Subject:  8
Loss:  9.5367431640625e-07
Subject:  9
Loss:  5.7220458984375e-05
Epoch:  47
Subject:  1
Loss:  0.0
Subject:  2
Loss:  0.0
Subject:  3
Loss:  2.09808349609375e-05
Subject:  4
Loss:  9.5367431640625e-06
Subject:  5
Loss:  9.5367431640625e-06
Subject:  6
Loss:  0.0
Subject:  7
Loss:  0.0018911361694335938
Subject:  8
Loss:  4.76837158203125e-06
Subject:  9
Loss:  0.04122018814086914
Epoch:  48
Subject:  1
Loss:  1.239776611328125e-05
Subject:  2
Loss:  0.0
Subject:  3
Loss:  9.5367431640625e-07
Subject:  4
Loss:  8.58306884765625e-06
Subject:  5
Loss:  0.0
Subject:  6
Loss:  4.76837158203125e-06
Subject:  7
Loss:  0.0
Subject:  8
Loss:  0.00019359588623046875
Subject:  9
Loss:  0.002258777618408203
Epoch:  49
Subject:  1
Loss:  1.9073486328125e-06
S

In [7]:
# Training Accuracy
predictions = []
labels = []
for k in range(X_train.shape[0]):
    X_train_i, y_train_i = X_train[k], y_train[k]
    print('Subject: ', k+1)
    for i in range(X_train_i.shape[0]):
        image, label = X_train_i[i], y_train_i[i]
        if np.any(np.isnan(image)):
            continue
        if cuda:
            image = autograd.Variable(torch.cuda.FloatTensor(image.reshape((image.shape[0],1,image.shape[1]))))
            label = autograd.Variable(torch.cuda.LongTensor([int(label %769)]))
        else:
            image = autograd.Variable(torch.FloatTensor(image.reshape((image.shape[0],1,image.shape[1]))))
            label = autograd.Variable(torch.LongTensor([int(label %769)]))
        optimizer.zero_grad()
        scores = cnn(image)
        scores = torch.mean(scores,0)
        scores = scores.view(1,4)
        prediction = np.argmax(scores.data.cpu().numpy())
        predictions.append(prediction)
        labels.append(int(label%769))
    diffs = [1 if prediction != label else 0 for prediction, label in zip(predictions, labels)]
    error = sum(diffs)/len(diffs)
    print("accuracy: {}".format(1 - error))

Subject:  1
accuracy: 0.9662447257383966
Subject:  2
accuracy: 0.9746300211416491
Subject:  3
accuracy: 0.9774330042313117
Subject:  4
accuracy: 0.9777777777777777
Subject:  5
accuracy: 0.9779661016949153
Subject:  6
accuracy: 0.9809187279151943
Subject:  7
accuracy: 0.9830611010284331
Subject:  8
accuracy: 0.9825119236883942
Subject:  9
accuracy: 0.9829867674858223


In [8]:
# Test Accuracy on all
for k in range(X_test.shape[0]):
    X_test_i, y_test_i = X_test[k], y_test[k]
    predictions = []
    labels = []
    print('Subject: ', k+1)
    for i in range(X_test_i.shape[0]):
        image, label = X_test_i[i], y_test_i[i]
        if np.any(np.isnan(image)):
            continue
        if cuda:
            image = autograd.Variable(torch.cuda.FloatTensor(image.reshape((image.shape[0],1,image.shape[1]))))
            label = autograd.Variable(torch.cuda.LongTensor([int(label %769)]))
        else:
            image = autograd.Variable(torch.FloatTensor(image.reshape((image.shape[0],1,image.shape[1]))))
            label = autograd.Variable(torch.LongTensor([int(label %769)]))
        optimizer.zero_grad()
        scores = cnn(image)
        scores = torch.mean(scores,0)
        scores = scores.view(1,4)
        prediction = np.argmax(scores.data.cpu().numpy())
        predictions.append(prediction)
        labels.append(int(label%769))


    diffs = [1 if prediction != label else 0 for prediction, label in zip(predictions, labels)]
    error = sum(diffs)/len(diffs)
    print("accuracy: {}".format(1 - error))

Subject:  1
accuracy: 0.4
Subject:  2
accuracy: 0.21999999999999997
Subject:  3
accuracy: 0.30000000000000004
Subject:  4
accuracy: 0.3125
Subject:  5
accuracy: 0.3829787234042553
Subject:  6
accuracy: 0.31999999999999995
Subject:  7
accuracy: 0.31999999999999995
Subject:  8
accuracy: 0.27083333333333337
Subject:  9
accuracy: 0.326530612244898
